In [16]:
import pandas as pd
import numpy as np
import os
os.chdir('/Users/sarwaridas/Desktop/IDS 702/Final Project/ids720_CoffeeAndCrimes/')
coffee_old=pd.read_csv("02_processed_data/coffee_by_year_CCA.csv")
coffee=pd.read_csv("02_processed_data/coffee_by_year_CCA_revised.csv")
crime=pd.read_csv("02_processed_data/crime_by_year_CCA.csv")

In [17]:
coffee_old=coffee_old.rename(columns={'keep_row':'coffeeshops'})
coffee_old=coffee_old.drop(['Unnamed: 0'],axis=1)
coffee_old=coffee_old.pivot_table(values="coffeeshops", index="CCA", columns="year").stack(dropna=False).reset_index()
coffee_old.head()

,CCA,year,0
0,Albany Park,2001.0,2.0
1,Albany Park,2002.0,22.0
2,Albany Park,2003.0,28.0
3,Albany Park,2004.0,32.0
4,Albany Park,2005.0,34.0


In [18]:
coffee=coffee.drop(['Unnamed: 0'],axis=1)
coffee=coffee.rename(columns={'keep_row':'coffeeshops'})
coffee['community']=coffee['community'].apply(lambda x: x.title())
coffee.head()



,community,year,coffeeshops
0,Albany Park,2002.0,11
1,Albany Park,2003.0,12
2,Albany Park,2004.0,17
3,Albany Park,2005.0,21
4,Albany Park,2006.0,20


In [19]:
# coffee=coffee.rename(columns={'keep_row':'coffeeshops'})
# coffee=coffee.drop(['Unnamed: 0'],axis=1)
# coffee=coffee.pivot_table(values="coffeeshops", index="CCA", columns="year").stack(dropna=False).reset_index()

In [20]:
lookup= pd.read_csv(os.getcwd()+"/02_processed_data/location_lookup.csv")
lookup= dict(zip(lookup.CCA, lookup.name))

In [21]:
crime['CCA_name']=crime["Community Area"].map(lookup)
crime['CCA_name']=crime['CCA_name'].apply(lambda x: x.title())

In [22]:
crime=crime.drop(['Unnamed: 0',"Community Area"],axis=1)
crime=crime.rename(columns={'ID':'crimes'})

crime.head()

,Year,crimes,CCA_name
0,2002,278,Rogers Park
1,2002,144,West Ridge
2,2002,202,Uptown
3,2002,63,Lincoln Square
4,2002,72,North Center


In [23]:
crime.nunique()

Year         20
crimes      474
CCA_name     77
dtype: int64

In [24]:
crime=crime.pivot_table(values="crimes", index="CCA_name", columns="Year").stack(dropna=False).reset_index()
crime.columns=['CCA_name', 'Year', 'crimes']


In [25]:
crime.head()

,CCA_name,Year,crimes
0,Albany Park,2002,100.0
1,Albany Park,2003,166.0
2,Albany Park,2004,161.0
3,Albany Park,2005,148.0
4,Albany Park,2006,134.0


In [26]:
coffee.head()


,community,year,coffeeshops
0,Albany Park,2002.0,11
1,Albany Park,2003.0,12
2,Albany Park,2004.0,17
3,Albany Park,2005.0,21
4,Albany Park,2006.0,20


In [27]:
df_yearly=crime.merge(coffee, how='outer', left_on=["Year",'CCA_name'], right_on=["year","community"],validate="1:1")


In [28]:
# df_yearly['coffeeshops']=df_yearly['coffeeshops'].interpolate()
df_yearly=df_yearly.interpolate()
df_yearly.drop(['community','year'],inplace=True,axis=1)
df_yearly.head(34)


,CCA_name,Year,crimes,coffeeshops
0,Albany Park,2002.0,100.0,11.000000
1,Albany Park,2003.0,166.0,12.000000
2,Albany Park,2004.0,161.0,17.000000
3,Albany Park,2005.0,148.0,21.000000
4,Albany Park,2006.0,134.0,20.000000
5,Albany Park,2007.0,135.0,23.000000
6,Albany Park,2008.0,116.0,4.000000
7,Albany Park,2009.0,112.0,23.000000
8,Albany Park,2010.0,130.0,8.000000
9,Albany Park,2011.0,129.0,20.000000


In [29]:
df_yearly.to_csv(os.getcwd()+"/02_processed_data/df_crimes_coffee_by_year.csv")

In [30]:
df_yearly.groupby(['CCA_name']).sum()

,Year,crimes,coffeeshops
CCA_name,,,
Albany Park,40230.0,2413.0,276.0
Archer Heights,40230.0,945.0,93.5
Armour Square,40230.0,1585.0,147.0
Ashburn,40230.0,2748.0,22.0
Auburn Gresham,40230.0,9143.0,42.0
...,...,...,...
West Lawn,40230.0,1674.0,34.0
West Pullman,40230.0,4243.0,43.0
West Ridge,40230.0,2844.0,262.0


In [32]:
#coffee.columns=['CCA', 'year', 'coffeeshops', 'timeperiod']

In [33]:
# df_full=crime.merge(coffee, how='outer', left_on=["Year",'CCA_name'], right_on=["year","CCA"])
# df_full.head()

In [34]:
#df_full.sample(10)

In [35]:
df_yearly.isna().sum()/len(df_yearly)

CCA_name       0.031447
Year           0.000000
crimes         0.000000
coffeeshops    0.000000
dtype: float64

In [50]:
df_yearly.sample(10)

,CCA_name,Year,crimes,coffeeshops,timeperiod
700,Jefferson Park,2002.0,26.0,2.000000,2005-2009
390,East Garfield Park,2012.0,347.0,1.000000,2010-2014
383,East Garfield Park,2005.0,273.0,1.000000,2005-2009
434,Edgewater,2016.0,93.0,9.000000,2015-2019
340,Douglas,2002.0,178.0,2.833333,2005-2009
913,Morgan Park,2015.0,65.0,1.000000,2015-2019
681,Irving Park,2003.0,162.0,15.000000,2005-2009
147,Avondale,2009.0,192.0,10.000000,2005-2009
1292,South Lawndale,2014.0,198.0,1.000000,2010-2014
558,Garfield Ridge,2020.0,44.0,4.000000,2020


In [39]:
df_yearly.head()

,CCA_name,Year,crimes,coffeeshops,timeperiod
0,Albany Park,2002.0,100.0,11.0,2005-2009
1,Albany Park,2003.0,166.0,12.0,2005-2009
2,Albany Park,2004.0,161.0,17.0,2005-2009
3,Albany Park,2005.0,148.0,21.0,2005-2009
4,Albany Park,2006.0,134.0,20.0,2005-2009


In [38]:
def category(x):
    if x <= 2009:
        return '2005-2009'
    elif x <= 2014: #& x>= 2010:
        return '2010-2014'
    elif x <= 2019:# & x>= 2015:
        return '2015-2019'
    elif x== 2020:
        return '2020'

# crime=crime.loc[crime.Year>=2005]
# crime=crime.loc[crime.Year<2021]

# coffee=coffee.loc[coffee.year>=2005]
# coffee=coffee.loc[coffee.year<2021]

# crime.Year=crime.Year.astype(int)
# coffee.year=coffee.year.astype(int)

df_yearly['timeperiod']=df_yearly.Year.apply(lambda x: category(x))
# coffee['timeperiod']=coffee.year.apply(lambda x: category(x))



In [43]:
df_by_timeperiod=df_yearly.groupby(['timeperiod','CCA_name']).sum().reset_index() #change this
df_by_timeperiod.drop("Year",inplace=True,axis=1)

In [49]:
df_by_timeperiod.sample(10)

,timeperiod,CCA_name,crimes,coffeeshops
275,2020,Montclare,14.0,2.0
27,2005-2009,Garfield Ridge,944.0,116.0
223,2015-2019,West Elsdon,239.0,6.5
191,2015-2019,Lake View,1059.0,217.0
186,2015-2019,Humboldt Park,1802.0,24.0
259,2020,Grand Boulevard,89.0,3.0
229,2015-2019,West Town,1611.0,216.0
176,2015-2019,Edison Park,11.0,12.0
148,2010-2014,West Garfield Park,1492.0,5.5
203,2015-2019,Near West Side,1789.0,252.0


In [45]:
df_by_timeperiod.to_csv(os.getcwd()+"/02_processed_data/coffee_crime_revised.csv")